In [5]:
import numpy as np
import random
from implementations import *
from helpers import load_csv_data


In [6]:
seed = 7

In [7]:
train_labels_raw, train_data_raw, train_ids = load_csv_data("data/train.csv")
_, test_data_raw, test_ids = load_csv_data("data/train.csv")


Code for visualisation of the data

In [67]:
import pandas as pd

df = pd.DataFrame(data=train_data_raw, index=None, columns=None)
df = df.applymap(lambda v: v if v > -999.0 else np.NaN)
print(len(df))
two_more_jets = df[(df[[22]] >= 2).to_numpy()].dropna()
one_jet = df[(df[[22]] == 1).to_numpy()].drop([4,5,6,12, 26,27,28], axis=1).dropna()
zero_jet = df[(df[[22]] == 0).to_numpy()].drop([4,5,6,12,23,24,25,26,27,28], axis=1).dropna()
print(len(two_more_jets)+len(one_jet)+len(zero_jet))
print(test_data_raw.shape)

250000
211886
(250000, 30)


## Data cleaning and grouping
This part creates three sets of rows depending on the number of jets

In [44]:
med_over = train_data_raw[train_data_raw[:,0] != -999]
med_DER_mass_MMC = np.median(med_over[:,0])

In [45]:
train_data = train_data_raw[train_data_raw[:,0] > -999].copy()
train_labels = train_labels_raw[train_data_raw[:,0] > -999].copy()
test_data = test_data_raw.copy()
test_data[test_data_raw[:,0] == -999] = med_DER_mass_MMC

In [46]:
two_more_jets = train_data[train_data[:,22] >= 2].copy()
one_jet = train_data[train_data[:,22] == 1].copy()
one_jet = np.delete(one_jet, np.s_[4,5,6,12,26,27,28], axis=1)
zero_jet = train_data[train_data[:,22] == 0].copy()
zero_jet = np.delete(zero_jet, np.s_[4,5,6,12,23,24,25,26,27,28], axis=1)

jet_sets = [zero_jet, one_jet, two_more_jets]

In [47]:
two_more_jets = train_labels[train_data[:,22] >= 2].copy()
one_jet = train_labels[train_data[:,22] == 1].copy()
zero_jet = train_labels[train_data[:,22] == 0].copy()

jet_sets_labels = [zero_jet, one_jet, two_more_jets]

In [48]:
two_more_jets_test = test_data[test_data[:,22] >= 2].copy()
one_jet_test = test_data[test_data[:,22] == 1].copy()
one_jet_test = np.delete(one_jet_test, np.s_[4,5,6,12,26,27,28], axis=1)
zero_jet_test = test_data[test_data[:,22] == 0].copy()
zero_jet_test = np.delete(zero_jet_test, np.s_[4,5,6,12,23,24,25,26,27,28], axis=1)


jet_sets_test = [zero_jet_test, one_jet_test, two_more_jets_test]

### Normalisation

In [49]:
for i in range(3):
    mean = jet_sets[i].mean(axis=0)
    std = jet_sets[i].std(axis=0)
    jet_sets[i] = (jet_sets[i] - mean)/std
    jet_sets[i][np.isnan(jet_sets[i])] = 0
    jet_sets_test[i] = (jet_sets_test[i] - mean)/std
    jet_sets_test[i][np.isnan(jet_sets_test[i])] = 0

/tmp/ipykernel_8166/428976276.py:4: RuntimeWarning: invalid value encountered in true_divide
  jet_sets[i] = (jet_sets[i] - mean)/std
/tmp/ipykernel_8166/428976276.py:6: RuntimeWarning: invalid value encountered in true_divide
  jet_sets_test[i] = (jet_sets_test[i] - mean)/std


## Applying the models to the test set

In [77]:
def make_predictions(test_sets, models):
    assert test_data.shape[0] == test_sets[0].shape[0] + test_sets[1].shape[0] + test_sets[2].shape[0]
    pred = np.zeros(test_data.shape[0])
    pred[test_data[:22] >= 2] = test_sets[2] @ models[2]
    pred[test_data[:22] == 1] = test_sets[1] @ models[1]
    pred[test_data[:22] == 0] = test_sets[0] @ models[0]
    decision = 0
    pred[np.where(pred <  decision)] = -1
    pred[np.where(pred >= decision)] = 1
    return pred
#make_predictions(jet_sets_test, [])

## Compute the models

functions to test different methods for regression

In [56]:
def compute_loss(y, tx, model):
    """
    compute the proportion of misprediction
    """
    pred = tx @ model
    decision = 0
    pred[np.where(pred <  decision)] = -1
    pred[np.where(pred >= decision)] = 1
    err = (pred - y)/2
    return np.absolute(err).mean()

def build_validation_sets(y, k_fold, seed):
    """
    returns indices for a train set and a test set
    """
    num_row = y.shape[0]
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    return indices[indices.shape[0]//(k_fold+1):], indices[:indices.shape[0]//(k_fold+1)],

def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree.
    
    Args:
        x: numpy array of shape (N,D)
        degree: integer.
        
    Returns:
        poly: numpy array of shape (N,D*d+1)
    """    
    N = x.shape[0]
    D = x.shape[1]
    #poly_base = np.zeros((N, D*degree + 1))
    poly_base = np.ones((N,1))
    poly_base = np.hstack((poly_base, x.copy()))
    for i in range(degree-1):
        range_start = 1 + i*D
        range_stop = 1 + (i+1)*D
        next_power = poly_base[:,range_start:range_stop]*x
        poly_base = np.hstack((poly_base, next_power))
    
    return poly_base

def cross_validation(y, x, train_model, degree):
    """
    Tests a certain training function using 4-fold cross-validation
    arguments: 
    train_model: func(y, tx) -> model
    returns: model, training_loss, test_loss
    """
    tx = build_poly(x, degree)
    k = 4
    train_idx, test_idx = build_validation_sets(x, k, seed)
    model, train_loss = train_model(y[train_idx], tx[train_idx])
    test_loss = compute_loss(y[test_idx], tx[test_idx], model)
    return model, train_loss, test_loss

Graph stuff

In [57]:
import numpy as np
import matplotlib.pyplot as plt

def cross_validation_visualization(lambds, mse_tr, mse_te):
    """visualization the curves of mse_tr and mse_te."""
    plt.semilogx(lambds, mse_tr, marker=".", color='b', label='train error')
    plt.semilogx(lambds, mse_te, marker=".", color='r', label='test error')
    plt.xlabel("lambda")
    plt.ylabel("r mse")
    #plt.xlim(1e-4, 1)
    plt.title("cross validation")
    plt.legend(loc=2)
    plt.grid(True)
    
def cross_validation_explore_lambda(train_model, degree, lambdas):
    """cross validation over regularisation parameter lambda.
    
    Args:
        train_model: func(y, tx, lambda) -> model
        degree: integer, degree of the polynomial expansion
        lambdas: shape = (p, ) where p is the number of values of lambda to test
    Returns:
        best_lambda : scalar, value of the best lambda
        best_mse : scalar, the associated mean squared error for the best lambda
    """
    seed = 12
    degree = degree
    lambdas = lambdas
    # define lists to store the loss of training data and test data
    mse_tr = []
    mse_te = []
    
    best_idx = 0
    idx = 0
    for lambda_ in np.nditer(lambdas):
        _, te, tr = cross_validation(lambda y,tx: train_model(y,tx, lambda_), degree)
        mse_tr.append(tr)
        mse_te.append(te)
        if te < mse_te[best_idx]:
            best_idx = idx
        idx += 1
    
    best_lambda = lambdas[best_idx]
    best_mse = mse_te[best_idx]
        
    cross_validation_visualization(lambdas, mse_tr, mse_te)
    print("For polynomial expansion up to degree %.f, the choice of lambda which leads to the best test mse is %.5f with a test mse of %.3f" % (degree, best_lambda, best_mse))
    return best_lambda, best_mse

lambdas = np.logspace(-4, 0, 30)
#best_lambda, best_rmse = cross_validation_demo(7, 4, np.logspace(-4, 0, 30))

Now try different methods

In [60]:
print(jet_sets_labels[0].shape,jet_sets[0].shape)
print((jet_sets[0].T @ jet_sets[0]).shape)

(73790,) (73790, 20)
(20, 20)


In [58]:
cross_validation(jet_sets_labels[0],jet_sets[0],least_squares,1)

LinAlgError: Singular matrix

In [ ]:
cross_validation_explore_lambda(lambda y,tx,lambda_: ridge_regression(y,tx,lambda_), 3, lambdas)

In [ ]:
cross_validation(lambda y,tx: logistic_regression(y, tx, np.zeros(tx.shape[1]), 100, 0.001), 1)